In [30]:
using BenchmarkTools
using Distances
using StaticArrays
using Plots
using LinearAlgebra
using ProgressBars
using CUDA
using Adapt

In [31]:
struct GaussianBeam{T<:Real}
    E0::T
    w0::T
    θ::T
    k::SVector{3, T}
    u2::SVector{3, T}
    u3::SVector{3, T}
end

GaussianBeam(E0, w0, θ) = GaussianBeam(E0, w0, θ, SVector(-sin(θ), zero(eltype(θ)), cos(θ)), SVector(cos(θ), zero(eltype(θ)), sin(θ)), SVector(zero(eltype(θ)), one(eltype(θ)), zero(eltype(θ))))

Adapt.@adapt_structure GaussianBeam

In [32]:
function optical_lattice!(Nd, Rd, a, d, scatterers, centered=false)
    scatterers[3, :] .= rand(0:(Nd-1)) .* d .+  a .* 2. .* (rand() .- 0.5) .-  a ./ 2.

    Na = size(scatterers, 2)
    u = rand(Na)
    theta = 2π * rand(Na)

    scatterers[1, :] .= Rd .* sqrt.(u) .* cos.(theta)
    scatterers[2, :] .= Rd .* sqrt.(u) .* sin.(theta)

    if centered
        scatterers[3, :] .-= (Nd - 1) * d / 2.0
    end

    return
end

function optical_lattice!(Nd, Rd, a, d, scatterers::CuDeviceMatrix{Float32, 1}, centered=false)
    i = (blockIdx().x - 1) * blockDim().x + threadIdx().x

    if i > size(scatterers, 2)
        return
    end

    u = rand(Float32)
    theta = 2.0f0π * rand(Float32)
    disk_no = rand(0:(Nd-1))

    @inbounds scatterers[1, i] = Rd * sqrt(u) * cos(theta)
    @inbounds scatterers[2, i] = Rd * sqrt(u) * sin(theta)
    @inbounds scatterers[3, i] = disk_no * d + a * 2.0f0 * (rand(Float32) - 0.5f0) - a / 2.0f0

    if centered
        @inbounds scatterers[3, i] -= (Nd - 1) * d / 2.0f0
    end

    return nothing
end

function bragg_periodicity(θ)
    return 1. / (2 * cos(θ))
end;

In [ ]:
function compute_system_matrix!(scatterers, M, Δ0)
    pairwise!(Euclidean(), M, scatterers, dims=2)

    @. M = cispi(2. * M) / (2.0π * 1.0im * M)

    for i in 1:Na
        @inbounds M[i, i] = 1.0 - 2.0im * Δ0
    end

    M .*= -0.5
    M = Symmetric(M)

    return
end

function compute_system_matrix!(scatterers::CuDeviceMatrix{Float32, 1}, M::CuDeviceMatrix{ComplexF32, 1}, Δ0::Float32)
    i = (blockIdx().x - 1) * blockDim().x + threadIdx().x
    j = (blockIdx().y - 1) * blockDim().y + threadIdx().y

    m, n = size(M)
    if i > m || j > n
        return nothing
    end

    if i == j
        @inbounds M[i, j] = -0.5f0 + 1.0f0im * Δ0
        return nothing
    end

    dist = zero(eltype(M))
    for k in 1:3
        @inbounds diff = scatterers[k, i] - scatterers[k, j]
        dist += diff * diff
    end

    @inbounds M[i, j] = sqrt(dist)
    @inbounds M[i, j] = -0.5f0 * exp(2.0f0im * π * M[i, j]) / (2.0f0im * π * M[i, j])

    return
end

In [ ]:
function compute_field(x, y, z; field::GaussianBeam{T})::ComplexF64 where T<:Real
    proj_z = field.k[1] * x + field.k[2] * y + field.k[3] * z

    proj_r2 = (field.u2[1] * x + field.u2[2] * y + field.u2[3] * z)^2 +
              (field.u3[1] * x + field.u3[2] * y + field.u3[3] * z)^2

    zR = π * field.w0^2
    wZ = field.w0 * sqrt(1 + (proj_z / zR)^2)
    phase = atan(proj_z / zR)

    if proj_z == 0
        return field.E0 * exp(-proj_r2 / field.w0^2)
    end

    Rz = proj_z * (1 + (zR / proj_z)^2)
    E = field.E0 * (field.w0 / wZ) * exp(-proj_r2 / wZ^2) * exp(2π * 1im * (proj_z + proj_r2 / (2 * Rz)) -1im * phase)
    return E
end

function compute_field(r; field)
    x, y, z = r
    return compute_field(x, y, z; field=field)
end;

function compute_field_xOz!(x1::Float32, x2::Float32, z1::Float32, z2::Float32, Np::Int32, field_matrix::CuDeviceMatrix{ComplexF32, 1})
    i = (blockIdx().x - 1) * blockDim().x + threadIdx().x
    j = (blockIdx().y - 1) * blockDim().y + threadIdx().y

    if i > Np || j > Np
        return nothing
    end

    δx = (x2 - x1) / (Np - 1)
    δz = (z2 - z1) / (Np - 1)

    x = x1 + (i - 1) * δx
    z = z1 + (j - 1) * δz

    Na::Int32 = size(scatterers, 2)
    for i in 1:Na
        @inbounds dist = (scatterers[1, i] - x)^2 + (scatterers[2, i])^2 + (scatterers[3, i] - z)^2
        @inbounds field_matrix[i, j] -= exp(2.0f0im * π * dist) / (2.0f0 * π * dist) * amplitudes[i]
    end

    return nothing
end

In [12]:
function compute_scattered_field(x, y, z; scatterers, amplitudes)
    result = ComplexF64(0.0)

    for i in axes(scatterers, 1)
        @inbounds dist = sqrt((scatterers[i, 1] - x )^2 + (scatterers[i, 2] - y)^2 + (scatterers[i, 3] - z)^2)
        @inbounds result -= amplitudes[i] * cispi(2 * dist) / (2.0π * dist)
    end

    return result
end

function compute_scattered_field_xOz(x1::Float32, x2::Float32, z1::Float32, z2::Float32, Np::Int32, field_matrix::CuDeviceMatrix{ComplexF32, 1}, scatterers::CuDeviceMatrix{Float32, 1}, amplitudes::CuDeviceVector{ComplexF32, 1})
    i = (blockIdx().x - 1) * blockDim().x + threadIdx().x
    j = (blockIdx().y - 1) * blockDim().y + threadIdx().y

    if i > Np || j > Np
        return nothing
    end

    δx = (x2 - x1) / (Np - 1)
    δz = (z2 - z1) / (Np - 1)

    x = x1 + (i - 1) * δx
    z = z1 + (j - 1) * δz

    Na::Int32 = size(scatterers, 2)
    for i in 1:Na
        @inbounds dist = (scatterers[1, i] - x)^2 + (scatterers[2, i])^2 + (scatterers[3, i] - z)^2
        @inbounds field_matrix[i, j] -= exp(2.0f0im * π * dist) / (2.0f0 * π * dist) * amplitudes[i]
    end

    return nothing
end

compute_scattered_field_xOz (generic function with 1 method)

In [33]:
println(BLAS.get_num_threads())
println(Threads.nthreads())

Na  = 2000
Nd  = 1
Rd  = 9
a   = 0.91
Δ0  = 0.05

E0  = 1e-3
w0  = 4.0
θ   = deg2rad(60)
d   = bragg_periodicity(deg2rad(60.0));

Np = 1000


6
12


1000

In [34]:
incident_field = GaussianBeam(E0, w0, θ)
scatterers = Matrix{Float32}(undef, 3, Na)
scatterers_d = CuMatrix{Float32}(undef, 3, Na)

M = Matrix{ComplexF32}(undef, Na, Na)
E = Vector{ComplexF32}(undef, Na)
A = Vector{ComplexF32}(undef, Na)

M_d = CuMatrix{ComplexF32}(undef, Na, Na)
E_d = CuVector{ComplexF32}(undef, Na)
field_d = CuMatrix{ComplexF32}(undef, Np, Np)

@benchmark CUDA.@sync begin
    @cuda threads=256 blocks=ceil(Int, Na / 256) optical_lattice!(Nd, Rd, a, d, scatterers_d)
end


BenchmarkTools.Trial: 10000 samples with 1 evaluation per sample.
 Range (min … max):  25.700 μs … 270.900 μs  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     39.600 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   42.673 μs ±  14.463 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

             █▅▃▁                                               
  ▅▇▇▆▄▄▄▅▃▃▂████▆▅▄▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▂▁▁▁▁▁▁▁▁▁▁▁▁ ▂
  25.7 μs         Histogram: frequency by time           91 μs <

 Memory estimate: 608 bytes, allocs estimate: 19.

In [25]:
@benchmark CUDA.@sync begin
    centered_optical_lattice!(Nd, Rd, a, d, scatterers)
    copyto!(scatterers_d, scatterers)
end

BenchmarkTools.Trial: 10000 samples with 1 evaluation per sample.
 Range (min … max):  388.100 μs …  66.054 ms  ┊ GC (min … max): 0.00% … 98.92%
 Time  (median):     452.150 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   483.036 μs ± 717.349 μs  ┊ GC (mean ± σ):  4.13% ±  5.16%

   ▁█▁                                                           
  ▁███▅▄▃▂▂▂▂▂▃▄▆▇▆▅▄▃▃▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁ ▂
  388 μs           Histogram: frequency by time          773 μs <

 Memory estimate: 234.66 KiB, allocs estimate: 14.

In [29]:
# compute_system_matrix!(scatterers, M, Δ0)

# threads = (16, 16)
# blocks = (cld(Na, threads[1]), cld(Na, threads[2]))

# CUDA.@sync begin
#     scatterers_d = CuArray{Float32}(scatterers)
#     CUDA.@cuda fastmath=true threads=threads blocks=blocks compute_system_matrix!(scatterers_d, M_d, Δ0)
# end

@benchmark begin
    centered_optical_lattice!(Nd, Rd, a, d, scatterers)

    threads = (16, 16)
    blocks = (cld(Na, threads[1]), cld(Na, threads[2]))

    CUDA.@sync begin
        scatterers_d = CuArray{Float32}(scatterers)
        CUDA.@cuda fastmath=true threads=threads blocks=blocks compute_system_matrix!(scatterers_d, M_d, Δ0)
    end

    E.= 0.5im .* compute_field.(@view(scatterers[1, :]), @view(scatterers[2, :]), @view(scatterers[3, :]), field=incident_field)
    copyto!(E_d, E);

    CUDA.@sync A_d = M_d \ E_d;

    threads = (16, 16)
    blocks = (cld(Np, threads[1]), cld(Np, threads[2]))

    CUDA.@sync CUDA.@cuda fastmath=true threads=threads blocks=blocks compute_field_xOz!(-60.0f0, 60.0f0, -60.0f0, 60.0f0, convert(Int32, Np), field_d, scatterers_d, A_d)
end

UndefVarError: UndefVarError: `compute_system_matrix!` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [34]:
E.= 0.5im .* compute_field.(@view(scatterers[1, :]), @view(scatterers[2, :]), @view(scatterers[3, :]), field=incident_field)
copyto!(E_d, E);

In [35]:
M = Symmetric(M)
A = M \ E;
A_d = M_d \ E_d;

In [44]:
threads = (16, 16)
blocks = (cld(Np, threads[1]), cld(Np, threads[2]))

CUDA.@cuda fastmath=true threads=threads blocks=blocks compute_field_xOz!(-60.0f0, 60.0f0, -60.0f0, 60.0f0, convert(Int32, Np), field_d, scatterers_d, A_d)

CUDA.HostKernel for compute_field_xOz!(Float32, Float32, Float32, Float32, Int32, CuDeviceMatrix{ComplexF32, 1}, CuDeviceMatrix{Float32, 1}, CuDeviceVector{ComplexF32, 1})